**Extracting product info from texts with LLM**

Patryk Prusak

#### Inputs & Consts

In [1]:
import json
from bert_score import score
from transformers import pipeline

Matplotlib is building the font cache; this may take a moment.
c:\Users\prusak.patryk\Documents\Repos\nlp-labs\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Define data

In [2]:
products = {
    "Thermal Mugs": {
        "reviews": [
            "The insulation is top-notch, keeps my drinks hot for hours with Contigo.",
            "I love the sleek design and the fact that it fits in my car's cup holder.",
            "The lid is a bit tricky to clean, but overall a great mug.",
            "It’s durable and doesn’t leak, even when tossed in my bag.",
            "Perfect for my morning coffee, keeps it hot until I finish it.",
            "Keeps my coffee hot for hours—just what I need for long workdays. Thanks, Contigo.",
            "The lid isn’t leak-proof, but it keeps drinks warm for a decent amount of time.",
            "Love the sleek design of my Zojirushi mug, and it fits perfectly in my car cup holder!",
            "It’s lightweight but keeps my drinks at the right temperature for hours with Hydro Flask.",
            "No more cold coffee! This Yeti thermal mug does the job.",
            "It’s easy to clean, and the thermal insulation works like a charm.",
            "The handle makes it easy to carry, and it doesn’t spill.",
            "Not great for keeping drinks cold, but excellent for hot beverages.",
            "I wish it were bigger, but it’s perfect for my morning tea.",
            "I accidentally dropped it, and it didn’t dent! Very sturdy.",
            "The rubber seal around the lid came loose after a few weeks. Disappointing.",
            "Great for both coffee and soup—keeps them warm for hours.",
            "The exterior stays cool, even when my drink is piping hot inside.",
            "I love the color options, and it’s great for on-the-go.",
            "Keeps ice water cold for hours, even in hot weather!",
            "It’s a little tricky to open one-handed, but overall, a great mug.",
            "The size is perfect for travel, and it keeps drinks hot all day.",
            "It doesn’t leak, even when I toss it in my bag. Highly recommend Contigo.",
            "The lid is a little tight, but the mug works well for keeping drinks warm.",
            "Very stylish and functional! I get compliments all the time.",
            "Keeps my coffee scalding hot for longer than any mug I’ve owned with Zojirushi.",
            "Great value for the price. Works just as well as more expensive brands.",
            "The mug is lightweight and easy to carry around.",
            "It fits perfectly under my single-serve coffee machine!",
            "Durable, sleek, and it does exactly what it’s supposed to.",
        ],
        "offers": [
            "Contigo Workday Travel Mug – Keeps Coffee Hot for Hours!",
            "Zojirushi Sleek Travel Mug – Perfect Fit for Car Holders",
            "Hydro Flask Lightweight Insulation Mug – Stay Warm for Hours",
            "Yeti Thermal Mug – No More Cold Coffee!",
            "Contigo All-Day Heat Retention Mug – Ideal for Travel",
            "Contigo Leak-Proof Mug – Toss in Your Bag with Confidence",
            "Zojirushi Scalding Hot Coffee Mug – Best Insulation Yet",
        ],
    },
    "Dishwasher Detergents": {
        "reviews": [
            "Cascade leaves my dishes spotless and smelling fresh.",
            "Finish Quantum is great for tough, greasy messes.",
            "I love how my glassware sparkles after using this detergent.",
            "It’s a bit pricey, but worth it for the clean results.",
            "No more pre-rinsing needed, this detergent does it all.",
            "My dishes come out sparkling clean every time with Cascade. Love this detergent!",
            "It works well on glass, but I’ve noticed spots on my silverware.",
            "Great for tough, greasy messes. Leaves no residue! Thanks, Finish.",
            "This detergent smells amazing and leaves my dishwasher fresh.",
            "It’s a little pricey, but my dishes have never looked better with Cascade Platinum.",
            "Gets rid of even the most stubborn baked-on food. Highly recommend Finish Quantum.",
            "Not the best on hard water stains, but otherwise it works great.",
            "My dishes have never been so spotless after a wash!",
            "It’s very effective, but I wish it came in a fragrance-free version.",
            "Cuts through grease like a dream. No more pre-rinsing with Cascade Complete.",
            "This detergent doesn’t leave any residue on plastic, which I love.",
            "My glasses come out clear and sparkling every single time.",
            "It doesn’t work well with my eco dishwasher. Dishes aren’t as clean.",
            "Very efficient—gets rid of food stains and smells with no problem.",
            "I noticed some streaks on my glassware, but overall it works well.",
            "Leaves my dishes spotless and my machine smelling fresh.",
            "A great, eco-friendly option that actually works!",
            "It’s a little hard on some of my delicate dishware.",
            "This is the only detergent that works on my hard water stains.",
            "No need to rewash dishes after using this—so efficient!",
            "Perfect for everyday use. My dishes are clean and shiny.",
            "Leaves a chemical smell, but it’s effective at cleaning.",
            "A bit expensive, but worth it for the spotless results.",
            "No more streaks or water spots! Best dishwasher detergent ever.",
            "My silverware and dishes look brand new after every wash.",
        ],
        "offers": [
            "Cascade Sparkling Clean Detergent – Your Dishes Will Shine",
            "Finish Detergent – Tough on Grease, No Residue Left",
            "Cascade Platinum Detergent – Pricey, But Worth It for Results",
            "Finish Quantum Detergent – Stubborn Food Stains Gone",
            "Cascade Complete Detergent – No Pre-Rinsing Needed for Grease",
        ],
    },
    "Sunscreens": {
        "reviews": [
            "Neutrogena sunscreen absorbs quickly and doesn’t leave a greasy residue.",
            "Perfect for sensitive skin, no breakouts or irritation.",
            "I love the lightweight formula, perfect for wearing under makeup.",
            "It’s waterproof, which is a must for pool days.",
            "A bit pricey, but the protection it provides is worth every penny.",
            "Absorbs quickly and doesn’t leave a greasy residue. Great for daily use with Neutrogena.",
            "The scent is a little strong, but it protects well with Coppertone.",
            "Perfect for sensitive skin! No breakouts or irritation with La Roche-Posay.",
            "A bit thick to apply, but once it’s on, it stays all day.",
            "I love the lightweight formula of CeraVe, perfect for wearing under makeup.",
            "Doesn’t leave a white cast, even on darker skin tones.",
            "The spray bottle makes it super easy to apply on the go.",
            "This sunscreen saved me from burning on a beach vacation with Banana Boat!",
            "It’s waterproof, which is a must for pool days. Highly recommend Hawaiian Tropic.",
            "A bit pricey, but the protection it provides is worth every penny with Supergoop.",
            "This is my go-to sunscreen for both my face and body with Neutrogena.",
            "It’s a little greasy, but it gets the job done in strong sun.",
            "No weird scent, and it goes on smooth. Love this product!",
            "Perfect for outdoor activities—no sunburn, even after hours outside.",
            "It’s great for kids! No irritation, and it’s easy to apply with Blue Lizard.",
            "A little too heavy for my face, but works perfectly for the body.",
            "The texture is nice and light, not sticky at all.",
            "This sunscreen doesn’t clog my pores, which is a huge plus with EltaMD.",
            "I’ve tried a lot of sunscreens, and this one offers the best protection with La Roche-Posay.",
            "It leaves a slight sheen, but I love how protected my skin feels.",
            "This formula doesn’t dry out my skin like others do.",
            "It’s great under makeup—no pilling or greasiness.",
            "I wish it were more affordable, but it’s worth it for the protection.",
            "Very effective, even after swimming for hours.",
            "My skin stays soft and protected all day with Neutrogena sunscreen.",
        ],
        "offers": [
            "Tide Powder Detergent – Fresh, Clean Clothes Every Time",
            "Ariel Powder Detergent – Whites Brighter, Even in Cold Water",
            "Persil Powder Detergent – Powerful Stain Removal",
            "OMO Powder Detergent – Leaves Residue on Dark Clothes",
            "Seventh Generation Powder Detergent – Great for Sensitive Skin",
            "Ecover Eco-Friendly Detergent – Perfect for the Eco-Conscious",
            "Gain Powder Detergent – Fresh-Smelling Laundry for Days",
            "Arm & Hammer Powder Detergent – Great Value, Lasts Forever",
            "Dreft Baby Powder Detergent – Gentle on Baby Clothes",
        ],
    },
    "Powder Detergents for Laundry": {
        "reviews": [
            "Tide gets my clothes fresh and clean every time.",
            "It dissolves well, even in cold water.",
            "Great for sensitive skin, no itching or redness.",
            "My laundry has never smelled so fresh, and it lasts for days.",
            "Perfect for my workout gear, gets rid of all the sweat smells.",
            "Gets my clothes fresh and clean every time. No lingering odor with Tide.",
            "It dissolves well, even in cold water. My whites have never been brighter thanks to Ariel.",
            "A little pricey, but worth it for the excellent stain removal power of Persil.",
            "This powder leaves a residue on darker clothes. Not a fan of OMO.",
            "Great for sensitive skin! No itching or redness after using Seventh Generation.",
            "I love how eco-friendly this detergent is. It’s a big plus for me with Ecover.",
            "I don’t need fabric softener anymore—this leaves my clothes so soft!",
            "My laundry has never smelled so fresh, and it lasts for days with Gain.",
            "It’s not the best for heavy stains but works great for daily washes.",
            "Great value for the price. This box lasts forever! Thanks, Arm & Hammer.",
            "Perfect for my workout gear—gets rid of all the sweat smells.",
            "Leaves a bit of powder behind in the machine, but it cleans well.",
            "I’ve been using it for years, and Tide never disappoints.",
            "Not as effective in hard water areas, but still decent.",
            "My go-to detergent for all of my family’s laundry needs.",
            "I noticed some fading in my darker clothes after a few washes.",
            "It’s gentle on my baby’s clothes and skin with Dreft.",
            "Very effective at removing mud and grass stains from the kids’ clothes.",
            "I like the scent, but it might be too strong for some.",
            "No complaints so far! My clothes feel clean and fresh.",
            "Works just as well as liquid detergents but at a lower cost.",
            "A bit too perfumed for my taste, but it gets the job done.",
            "My clothes are noticeably softer and smell better than before.",
            "The box is hard to pour from, but the detergent works well.",
            "This is my new favorite detergent. So much better than the leading brand!",
        ],
        "offers": [
            "Neutrogena Daily Sunscreen – Quick Absorption, No Grease",
            "Coppertone Suncream – Strong Scent, Strong Protection",
            "La Roche-Posay Sensitive Skin Sunscreen – No Breakouts",
            "CeraVe Lightweight Sunscreen – Perfect Under Makeup",
            "Banana Boat Beach-Saver Sunscreen – No Burns, Just Fun",
            "Hawaiian Tropic Waterproof Sunscreen – Pool Day Essential",
            "Supergoop Premium Sunscreen – Worth Every Penny",
            "Neutrogena Face & Body Sunscreen – All-Purpose Protection",
            "EltaMD Pore-Friendly Sunscreen – Protection Without Clogging",
            "La Roche-Posay Suncream – The Best in Sun Protection",
            "Neutrogena Sunscreen – Soft, Protected Skin All Day",
        ],
    },
}

In [3]:
prompts = {
    "review1": {
        "review_content": "Gets my clothes fresh and clean every time. No lingering odor with Tide.",
        "golden_answer": {
            "product category": "Powder Detergents for Laundry",
            "brand": "Tide",
            "other keywords": ["fresh", "clean", "no lingering odor"],
        },
    },
    "review2": {
        "review_content": "Keeps my coffee hot for hours—just what I need for long workdays. Thanks, Contigo.",
        "golden_answer": {
            "product category": "Thermal Mugs",
            "brand": "Contigo",
            "other keywords": ["hot for hours"],
        },
    },
    "review3": {
        "review_content": "This vacuum cleaner is lightweight and powerful. Makes cleaning a breeze. Thanks, Dyson.",
        "golden_answer": {
            "product category": "Vacuum Cleaners",
            "brand": "Dyson",
            "other keywords": ["lightweight", "powerful", "cleaning a breeze"],
        },
    },
    "review4": {
        "review_content": "The screen is crystal clear and the battery lasts all day. Love my new iPhone.",
        "golden_answer": {
            "product category": "Smartphones",
            "brand": "Apple",
            "other keywords": ["crystal clear screen", "battery lasts all day"],
        },
    },
    "review5": {
        "review_content": "This blender crushes ice and blends smoothies perfectly. Great job, Ninja.",
        "golden_answer": {
            "product category": "Blenders",
            "brand": "Ninja",
            "other keywords": ["crushes ice", "blends smoothies perfectly"],
        },
    },
}

#### Models Inference Definition

Comparison metrics

In [4]:
def compare_product_review_similarity(review_data, product_data):
    """
    Calculates a similarity score between a review and a product based on
    categories, brands, and keywords, including a comparison of full product
    title with review information using BERTScore.
    """

    similarity_score = 0

    # Category Matching (Highest weight)
    if review_data.get("product category") == product_data.get("product category"):
        similarity_score += 0.5

    # Brand Matching (Medium weight)
    if review_data.get("brand") == product_data.get("brand"):
        similarity_score += 0.3

    # String Comparison (BERTScore) between Product Title and Review Data
    review_info_string = " ".join(
        [
            review_data.get("product category"),
            review_data.get("brand"),
            " ".join(review_data.get("other keywords")),
        ]
    )

    product_info_string = " ".join(
        [
            product_data.get("product category"),
            product_data.get("brand"),
            " ".join(product_data.get("other keywords")),
        ]
    )

    P, R, F1 = score(
        [product_info_string],
        [review_info_string],
        lang="en",
        model_type="bert-base-uncased",
        verbose=False,
    )

    print(F1.mean().item())
    similarity_score += F1.mean().item() * 0.2

    return round(min(1, similarity_score) * 100)


def compare_keywords_bert_score(review_keywords, product_keywords):
    """Compares keywords using BERTScore."""

    if not review_keywords or not product_keywords:
        return 0
    print(review_keywords)
    print(product_keywords)

    # Calculate BERTScore
    P, R, F1 = score(
        [keyword for keyword in review_keywords],
        [keyword for keyword in product_keywords],
        lang="en",
        model_type="bert-base-uncased",
        verbose=False,
    )

    return F1.mean().item()

In [5]:
# define prompt template
prompt_template = """
Extract information from the following reviews:
{content}

Return json format with the following JSON schema:

{{
        "product category": {{
            "type": "string",
            "enum": ["Powder Detergents for Laundry", "Thermal Mugs", "Dishwasher Detergents", "Sunscreens", "Nappies", "Others"]
        }},
        "brand": {{
            "type": "string" or N/A
        }},
        "other keywords": {{
            "type": "array",
            "items": {{
                "type": "string"
            }}
        }},

}}
"""

#### Execution

In [ ]:
# Initialize the pipeline with a suitable model
nlp = pipeline("text-generation", model="meta-llama/Meta-Llama-3-8B-Instruct")

# Prepare the reviews content
reviews_content = products["Sunscreens"]["reviews"][0]

# Format the prompt
prompt = prompt_template.format(content=reviews_content)
messages = [
    {"role": "user", "content": prompt},
]
# Generate the response
response = nlp(messages, max_length=1024, num_return_sequences=1)

# Print the generated response
print(response[0]["generated_text"])

c:\Users\prusak.patryk\Documents\Repos\nlp-labs\.venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\prusak.patryk\.cache\huggingface\hub\models--meta-llama--Meta-Llama-3-8B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shards: 100%|██████████| 4/4 [01:48<00:00, 27.22s/it

In [ ]:
review = "Absorbs quickly and doesn’t leave a greasy residue. Great for daily use SPf50 with Neutrogena."
product = "Neutrogena SPF50 Pore-Friendly Sunscreen – Protection Without Clogging"
review_data = {
    "product category": "Sunscreens",
    "brand": "Neutrogena",
    "other keywords": ["spf50", "daily use"],
}
product_data = {
    "product category": "Sunscreens",
    "brand": "Neutrogena",
    "other keywords": ["spf50", "Pore-Friendly", "Protection", "daily use"],
}

similarity_score_new = compare_product_review_similarity(review_data, product_data)

print(
    f"New Similarity between review:\n '{review}'\n with data'{review_data}'\n and product:\n '{product}'\n with data'{product_data}''\n is as follow:\n {similarity_score_new}"
)

#### Results